In [15]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Trees').getOrCreate()

In [17]:
df = spark.read.csv('data.csv', inferSchema=True, header=True)

In [19]:
df.printSchema()

root
 |-- private: string (nullable = true)
 |-- apps: integer (nullable = true)
 |-- accept: integer (nullable = true)
 |-- enroll: integer (nullable = true)
 |-- top10perc: integer (nullable = true)
 |-- top25perc: integer (nullable = true)
 |-- f_undergrad: integer (nullable = true)
 |-- p_undergrad: integer (nullable = true)
 |-- outstate: integer (nullable = true)
 |-- room_board: integer (nullable = true)
 |-- books: integer (nullable = true)
 |-- personal: integer (nullable = true)
 |-- phd: integer (nullable = true)
 |-- terminal: integer (nullable = true)
 |-- s_f_ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- expend: integer (nullable = true)
 |-- grad_rate: integer (nullable = true)



In [20]:
df.columns

['private',
 'apps',
 'accept',
 'enroll',
 'top10perc',
 'top25perc',
 'f_undergrad',
 'p_undergrad',
 'outstate',
 'room_board',
 'books',
 'personal',
 'phd',
 'terminal',
 's_f_ratio',
 'perc_alumni',
 'expend',
 'grad_rate']

In [22]:
from pyspark.ml.feature import VectorAssembler, StringIndexer
assembler = VectorAssembler(
    inputCols=[
        'apps',
        'accept',
        'enroll',
        'top10perc',
        'top25perc',
        'f_undergrad',
        'p_undergrad',
        'outstate',
        'room_board',
         'books',
         'personal',
         'phd',
         'terminal',
         's_f_ratio',
         'perc_alumni',
         'expend',
         'grad_rate'], 
    outputCol='features')
stridx = StringIndexer(inputCol='private', outputCol='label')
final_data = assembler.transform(df)
final_data = stridx.fit(final_data).transform(final_data)

In [23]:
from pyspark.ml.classification import (RandomForestClassifier, GBTClassifier,
                                       DecisionTreeClassifier)

In [24]:
train, test = final_data.randomSplit([0.7, 0.3])

In [25]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees = 100)
gbt = GBTClassifier()

In [26]:
dtc_model = dtc.fit(train)
rfc_model = rfc.fit(train)
gbt_model = gbt.fit(train)

In [27]:
dtc_preds = dtc_model.transform(test)
rfc_preds = rfc_model.transform(test)
gbt_preds = gbt_model.transform(test)

In [9]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[122,123,148...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[123,124,125...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[125,126,127...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[150,151,152...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[151,152,153...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[153,154,155...|   [29.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(69

In [28]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
evaluator = MulticlassClassificationEvaluator(metricName='accuracy')

In [29]:
print(f'DTC: {evaluator.evaluate(dtc_preds)}') 
print(f'RFC: {evaluator.evaluate(rfc_preds)}')
print(f'GBT: {evaluator.evaluate(gbt_preds)}')

DTC: 0.904
RFC: 0.936
GBT: 0.908
